
# EEF Quant – Week 3  
## Stochastic Processes for Quant Finance

This notebook covers:

- Discrete **Random Walk**
- **Brownian Motion** \(W_t\)
- **Geometric Brownian Motion (GBM)** for stock prices
- **Ornstein–Uhlenbeck (OU)** mean-reverting process
- A **GBM Monte Carlo** experiment



In [ ]:

import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (9, 4)
plt.rcParams['axes.grid'] = True

np.random.seed(42)



## 1. Discrete Random Walk

A simple random walk is defined by:

\[
X_{t+1} = X_t + \varepsilon_t, \quad \varepsilon_t \sim \mathcal{N}(0, \sigma^2)
\]

Used as a toy model for cumulative shocks or log-prices.


In [ ]:

def simulate_random_walk(n_steps=500, sigma=1.0):
    eps = np.random.normal(0.0, sigma, n_steps)
    return np.cumsum(eps)

# Single path
rw = simulate_random_walk(n_steps=500, sigma=1.0)

plt.plot(rw)
plt.title("Random Walk – Single Path")
plt.xlabel("Step")
plt.ylabel("X_t")
plt.show()

# Multiple paths
n_paths = 10
paths = np.zeros((500, n_paths))
for i in range(n_paths):
    paths[:, i] = simulate_random_walk(n_steps=500, sigma=1.0)

plt.plot(paths)
plt.title("Random Walk – 10 Paths")
plt.xlabel("Step")
plt.ylabel("X_t")
plt.show()



## 2. Brownian Motion \(W_t\)

Brownian motion is the continuous-time limit of the random walk.

Properties:

- \(W_0 = 0\)
- Continuous paths
- Independent increments
- \(W_t - W_s \sim \mathcal{N}(0, t-s)\)

We simulate it by:

\[
W_{t+\Delta t} = W_t + \sqrt{\Delta t}\, Z_t, \quad Z_t \sim \mathcal{N}(0,1)
\]


In [ ]:

def simulate_brownian_motion(T=1.0, n_steps=252):
    dt = T / n_steps
    increments = np.random.normal(0.0, np.sqrt(dt), size=n_steps)
    W = np.cumsum(increments)
    t = np.linspace(dt, T, n_steps)
    return t, W

t_bm, W = simulate_brownian_motion(T=1.0, n_steps=252)

plt.plot(t_bm, W)
plt.title("Brownian Motion $W_t$")
plt.xlabel("Time (years)")
plt.ylabel("W_t")
plt.show()



## 3. Geometric Brownian Motion (GBM)

Classical stock price model:

\[
dS_t = \mu S_t \, dt + \sigma S_t \, dW_t
\]

- \(\mu\): drift (expected return)  
- \(\sigma\): volatility  

Discrete approximation:

\[
S_{t+\Delta t} = S_t \exp\left((\mu - \tfrac{1}{2}\sigma^2)\Delta t + \sigma \sqrt{\Delta t} Z_t\right)
\]


In [ ]:

def simulate_gbm_path(S0=100, mu=0.08, sigma=0.2, T=1.0, n_steps=252):
    dt = T / n_steps
    Z = np.random.normal(0.0, 1.0, size=n_steps)
    S = np.zeros(n_steps + 1)
    S[0] = S0
    for t in range(1, n_steps + 1):
        S[t] = S[t-1] * np.exp(
            (mu - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z[t-1]
        )
    time_grid = np.linspace(0, T, n_steps + 1)
    return time_grid, S

# Single path
t_gbm, S_gbm = simulate_gbm_path()

plt.plot(t_gbm, S_gbm)
plt.title("GBM Stock Price Path (1 year)")
plt.xlabel("Time (years)")
plt.ylabel("S_t")
plt.show()

# Multiple paths
def simulate_gbm_paths(S0=100, mu=0.08, sigma=0.2, T=1.0, n_steps=252, n_paths=20):
    dt = T / n_steps
    time_grid = np.linspace(0, T, n_steps + 1)
    paths = np.zeros((n_steps + 1, n_paths))
    paths[0, :] = S0
    for i in range(n_paths):
        Z = np.random.normal(0.0, 1.0, size=n_steps)
        for t in range(1, n_steps + 1):
            paths[t, i] = paths[t-1, i] * np.exp(
                (mu - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z[t-1]
            )
    return time_grid, paths

t_grid, S_paths = simulate_gbm_paths()

plt.plot(t_grid, S_paths)
plt.title("GBM – 20 Paths")
plt.xlabel("Time (years)")
plt.ylabel("S_t")
plt.show()



## 4. Ornstein–Uhlenbeck (OU) Mean-Reverting Process

Mean-reversion model:

\[
dX_t = \theta(\mu - X_t)dt + \sigma dW_t
\]

- \(\mu\): long-run mean  
- \(\theta\): speed of mean-reversion  
- \(\sigma\): volatility of shocks  


In [ ]:

def simulate_ou_process(X0=1.0, mu=0.0, theta=1.5, sigma=0.3, T=1.0, n_steps=252):
    dt = T / n_steps
    X = np.zeros(n_steps + 1)
    X[0] = X0
    for t in range(1, n_steps + 1):
        dX = theta * (mu - X[t-1]) * dt + sigma * np.sqrt(dt) * np.random.normal()
        X[t] = X[t-1] + dX
    time_grid = np.linspace(0, T, n_steps + 1)
    return time_grid, X

t_ou, X_ou = simulate_ou_process()

plt.plot(t_ou, X_ou)
plt.axhline(0.0, linestyle='--', label='mean')
plt.title("OU Mean-Reverting Process")
plt.xlabel("Time (years)")
plt.ylabel("X_t")
plt.legend()
plt.show()



## 5. GBM Monte Carlo – Distribution of Final Prices

We now simulate many GBM paths and examine the distribution of final prices \(S_T\).

Parameters example:

- \(S_0 = 100\)
- \(\mu = 8\%\)
- \(\sigma = 20\%\)
- \(T = 1\) year
- 252 steps
- 1000 paths


In [ ]:

def gbm_monte_carlo(S0=100, mu=0.08, sigma=0.2,
                    T=1.0, n_steps=252, n_paths=1000):
    dt = T / n_steps
    time_grid = np.linspace(0, T, n_steps + 1)
    paths = np.zeros((n_steps + 1, n_paths))
    paths[0, :] = S0
    for i in range(n_paths):
        Z = np.random.normal(0.0, 1.0, size=n_steps)
        for t in range(1, n_steps + 1):
            paths[t, i] = paths[t-1, i] * np.exp(
                (mu - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z[t-1]
            )
    return time_grid, paths

t_mc, S_mc_paths = gbm_monte_carlo()
S_T = S_mc_paths[-1, :]

plt.hist(S_T, bins=40, density=True)
plt.title("GBM Monte Carlo – Distribution of $S_T$")
plt.xlabel("S_T")
plt.ylabel("Density")
plt.show()

mean_ST = np.mean(S_T)
std_ST = np.std(S_T)
prob_below_90 = np.mean(S_T < 90)
prob_above_120 = np.mean(S_T > 120)

log_returns = np.log(S_T / 100.0)
expected_return = np.mean(np.exp(log_returns) - 1)
vol_of_returns = np.std(log_returns)

print(f"Mean S_T: {mean_ST:.2f}")
print(f"Std(S_T): {std_ST:.2f}")
print(f"P(S_T < 90):  {prob_below_90:.3f}")
print(f"P(S_T > 120): {prob_above_120:.3f}")
print(f"Approx expected return: {expected_return:.2%}")
print(f"Vol of log-returns: {vol_of_returns:.2%}")
